<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao27_IMDB_BERT_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Fine-Tuning do BERT para Análise de Sentimentos (IMDb)

Este notebook demonstra o processo de fine-tuning de um modelo BERT pré-treinado para a tarefa de classificação de sentimentos (positivo ou negativo) utilizando o dataset de reviews de filmes do IMDb.

Usaremos as bibliotecas da Hugging Face (`transformers`, `datasets` e `evaluate`) para simplificar o processo.

## Passo 1: Instalação e Configuração

Primeiro, instalamos as bibliotecas necessárias.

In [2]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [3]:
# Importações principais
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate

## Passo 2: Carregamento do Dataset (IMDb)

Vamos carregar o dataset IMDb diretamente da biblioteca `datasets`. Ele já vem convenientemente dividido em `train` e `test`.

**Nota:** O dataset IMDb é grande. Para acelerar o processo de fine-tuning neste exemplo, usaremos um subconjunto menor (5000 para treino, 1000 para teste). Para um resultado final robusto, remova os comandos `.select()`.

In [4]:
# Carrega o dataset IMDb
dataset = load_dataset("imdb")

# Para um treinamento mais rápido (demonstração), vamos usar um subconjunto
# Remova as duas linhas abaixo para treinar no dataset completo
train_dataset = dataset["train"].shuffle(seed=42).select(range(5000))
test_dataset = dataset["test"].shuffle(seed=42).select(range(1000))

# Se quiser usar o dataset completo (pode levar horas sem uma GPU premium):
# train_dataset = dataset["train"]
# test_dataset = dataset["test"]

print(f"Exemplos de treino: {len(train_dataset)}")
print(f"Exemplos de teste: {len(test_dataset)}")
print("\nExemplo de dado:")
print(train_dataset[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Exemplos de treino: 5000
Exemplos de teste: 1000

Exemplo de dado:
{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}


## Passo 3: Carregamento do Tokenizador e Modelo

Vamos usar o `bert-base-uncased`, um modelo BERT padrão (em inglês, *uncased* significa que ele não diferencia maiúsculas de minúsculas).

1.  **Tokenizer**: Responsável por converter o texto em números (IDs) que o BERT entende.
2.  **Model**: Usamos `AutoModelForSequenceClassification`. Isso carrega o BERT com uma "cabeça" de classificação de sequência no topo, pronta para o fine-tuning.

In [5]:
# Checkpoint do modelo pré-treinado
model_checkpoint = "bert-base-uncased"

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Carrega o modelo de classificação de sequência
# num_labels=2 (negativo e positivo)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Passo 4: Pré-processamento (Tokenização)

Precisamos aplicar o tokenizador a todos os textos do dataset.

Criamos uma função `tokenize_function` que aplica o tokenizador. Usamos `truncation=True` para garantir que reviews muito longos sejam cortados no limite máximo do BERT (512 tokens).

In [6]:
def tokenize_function(examples):
    # 'truncation=True' corta textos maiores que o limite máximo do modelo (512 para o BERT)
    return tokenizer(examples["text"], truncation=True, max_length=512)

# Aplica a tokenização aos datasets usando .map() (é rápido e paralelizado)
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# O DataCollator cuidará do padding dinamicamente (mais eficiente)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Passo 5: Definição da Métrica de Avaliação

Durante o treinamento, queremos monitorar o desempenho do modelo no set de validação (neste caso, usaremos o `test_dataset` para isso). A métrica padrão para classificação é a acurácia.

In [7]:
# Carrega a métrica de acurácia
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Converte logits (saída bruta do modelo) em previsões (a classe com maior score)
    predictions = np.argmax(logits, axis=-1)

    # Compara previsões com os labels reais
    return accuracy_metric.compute(predictions=predictions, references=labels)

## Passo 6: Configuração do Treinamento (Fine-Tuning)

Agora, configuramos o `Trainer`, que é a classe da Hugging Face que gerencia todo o processo de fine-tuning.

1.  **`TrainingArguments`**: Define todos os hiperparâmetros do treinamento (taxa de aprendizado, número de épocas, tamanho do batch, etc.).
2.  **`Trainer`**: Recebe o modelo, os argumentos, os datasets, o tokenizador, o *data collator* e a função de métrica.

In [8]:
# Define os argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./bert-imdb-finetuned",    # Onde salvar o modelo
    learning_rate=2e-5,                    # Taxa de aprendizado (comum para fine-tuning de BERT)
    per_device_train_batch_size=8,         # Tamanho do batch de treino (ajuste conforme a VRAM da sua GPU)
    per_device_eval_batch_size=8,          # Tamanho do batch de avaliação
    num_train_epochs=3,                    # Número de épocas de treinamento
    weight_decay=0.01,                     # Regularização
    eval_strategy="epoch",           # Avaliar o modelo no final de cada época
    save_strategy="epoch",                 # Salvar o modelo no final de cada época
    load_best_model_at_end=True,           # Carregar o melhor modelo ao final do treino
    push_to_hub=False,                     # Não fazer upload para o Hugging Face Hub
)

# Cria o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,  # Usamos o 'test' como validação aqui
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2349608138.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Passo 7: Iniciar o Treinamento

Este é o passo principal. Basta chamar `trainer.train()`. Se você estiver no Colab com uma GPU (Vá em *Ambiente de execução > Alterar tipo de ambiente de execução > Acelerador de hardware > T4 GPU*), isso deve ser relativamente rápido.

In [9]:
# Inicia o fine-tuning!
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jsansao to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.365900,0.260296,0.906000
2,0.222800,0.397173,0.908000
3,0.113400,0.455603,0.912000


TrainOutput(global_step=1875, training_loss=0.19815029398600262, metrics={'train_runtime': 1600.3751, 'train_samples_per_second': 9.373, 'train_steps_per_second': 1.172, 'total_flos': 3708801103023840.0, 'train_loss': 0.19815029398600262, 'epoch': 3.0})

## Passo 8: Avaliação Final

Após o término do treinamento, o `trainer` carregou o melhor checkpoint (baseado na `evaluation_strategy`). Podemos rodar uma avaliação final no dataset de teste para ver o resultado.

In [10]:
# Avalia o melhor modelo no dataset de teste
eval_results = trainer.evaluate()

print("\nResultados da Avaliação Final:")
print(eval_results)


Resultados da Avaliação Final:
{'eval_loss': 0.2602960169315338, 'eval_accuracy': 0.906, 'eval_runtime': 29.5189, 'eval_samples_per_second': 33.877, 'eval_steps_per_second': 4.235, 'epoch': 3.0}


## Passo 9: Teste com Novas Frases (Inferência)

Vamos usar a ferramenta `pipeline` da Hugging Face para testar nosso modelo recém-treinado com frases personalizadas.

**Importante:** O modelo foi treinado no dataset IMDb (inglês) e usou o `bert-base-uncased` (inglês). Portanto, ele só funcionará bem para textos em inglês.

In [11]:
from transformers import pipeline

# Detecta se há GPU disponível
device = 0 if torch.cuda.is_available() else -1

# Carrega o pipeline de análise de sentimentos com nosso melhor modelo salvo
best_model_path = trainer.state.best_model_checkpoint
print(f"\nCarregando o melhor modelo salvo de: {best_model_path}")

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=best_model_path,
    tokenizer=tokenizer,
    device=device  # -1 para CPU, 0 para GPU
)

# --- Teste com frases em INGLÊS ---
print("\n--- Teste em Inglês ---")
review_pos = "This movie was fantastic! The acting was superb and the plot was engaging."
review_neg = "I hated this film. It was boring and the actors were terrible."

# O modelo IMDb usa LABEL_1 para positivo e LABEL_0 para negativo
print(f"Frase: {review_pos}")
print(f"Resultado: {sentiment_pipeline(review_pos)}")

print(f"\nFrase: {review_neg}")
print(f"Resultado: {sentiment_pipeline(review_neg)}")


# --- (Opcional) Teste em Português ---
print("\n--- Teste em Português (Opcional) ---")
print("AVISO: O modelo foi treinado em INGLÊS. O desempenho em português será muito baixo.")

review_pt_pos = "Eu amei este filme, foi incrível!"
print(f"\nFrase: {review_pt_pos}")
print(f"Resultado: {sentiment_pipeline(review_pt_pos)}")


Carregando o melhor modelo salvo de: ./bert-imdb-finetuned/checkpoint-625


Device set to use cuda:0



--- Teste em Inglês ---
Frase: This movie was fantastic! The acting was superb and the plot was engaging.
Resultado: [{'label': 'LABEL_1', 'score': 0.9934056997299194}]

Frase: I hated this film. It was boring and the actors were terrible.
Resultado: [{'label': 'LABEL_0', 'score': 0.9912347793579102}]

--- Teste em Português (Opcional) ---
AVISO: O modelo foi treinado em INGLÊS. O desempenho em português será muito baixo.

Frase: Eu amei este filme, foi incrível!
Resultado: [{'label': 'LABEL_1', 'score': 0.6833258867263794}]


**Para análise de sentimentos em português**, você deveria usar um modelo pré-treinado em português (como `neuralmind/bert-base-portuguese-cased`) e um dataset em português (como o [B2W-Reviews](https://huggingface.co/datasets/b2w-reviews) ou [Olist Reviews](https://huggingface.co/datasets/olist_user_order_reviews)).